### ⽤于预训练BERT的数据集

In [11]:
import os
import random
import torch
from d2l import torch as d2l

In [7]:
d2l.DATA_HUB['wikitext-2'] = (
    'https://s3.amazonaws.com/research.metamind.io/wikitext/'
    'wikitext-2-v1.zip', '3c914d17d80b1459be871a5039ac23e752a53cbe')


def _read_wiki(data_dir):
    file_name = os.path.join(data_dir, 'wiki.train.tokens')
    with open(file_name, 'r', encoding='utf-8') as f:
        lines = f.readlines()
        # 大写字母转换为小写字母
    paragraphs = [line.strip().lower().split(' . ') for line in lines if len(line.split(' . ')) >= 2]
    random.shuffle(paragraphs)
    return paragraphs

#### 为预训练任务定义辅助函数

**我们⾸先为BERT的两个预训练任务实现辅助函数。这些辅助函数将在稍后将原始⽂本语料库转
换为理想格式的数据集时调⽤，以预训练BERT。**

#### ⽣成下⼀句预测任务的数据

In [18]:
def _get_next_sentence(sentence, next_sentence, paragraphs):
    if random.random() < 0.5:
        is_next = True
    else:
        # paragraphs是三重列表的嵌套
        next_sentence = random.choice(random.choice(paragraphs))
        is_next = False
    return sentence, next_sentence, is_next

In [16]:
def _get_nsp_data_from_paragraph(paragraph, paragraphs, vocab, max_len):
    nsp_data_from_paragraph = []
    for i in range(len(paragraph) - 1):
        tokens_a, tokens_b, is_next = _get_next_sentence(paragraph[i], paragraph[i + 1], paragraphs)
        # 考虑一个'<cls>'词元和2个'<sep>'词元
        if len(tokens_a) + len(tokens_b) + 3 > max_len:
            continue
        tokens, segments = d2l.get_tokens_and_segments(tokens_a, tokens_b)
        nsp_data_from_paragraph.append((tokens, segments, is_next))
    return nsp_data_from_paragraph

#### ⽣成遮蔽语⾔模型任务的数据

**为了从BERT输⼊序列⽣成遮蔽语⾔模型的训练样本，我们定义了以下_replace_mlm_tokens函数。在其
输⼊中，tokens是表⽰BERT输⼊序列的词元的列表，candidate_pred_positions是不包括特殊词元
的BERT输⼊序列的词元索引的列表（特殊词元在遮蔽语⾔模型任务中不被预测），以及num_mlm_preds指
⽰预测的数量（选择15%要预测的随机词元）。**
**定义遮蔽语⾔模型任务之后，在每个预测位置，
输⼊可以由特殊的“掩码”词元或随机词元替换，或者保持不变。最后，该函数返回可能替换后的输⼊词元、
发⽣预测的词元索引和这些预测的标签。**

In [22]:
def _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                        vocab):
    # 为遮蔽语⾔模型的输⼊创建新的词元副本，其中输⼊可能包含替换的“<mask>”或随机词元
    mlm_input_tokens = [token for token in tokens]
    pred_positions_and_labels = []
    # 打乱后⽤于在遮蔽语⾔模型任务中获取15%的随机词元进⾏预测
    random.shuffle(candidate_pred_positions)
    for mlm_pred_position in candidate_pred_positions:
        if len(pred_positions_and_labels) >= num_mlm_preds:
            break
        masked_token = None
        # 80%的时间：将词替换为“<mask>”词元
        if random.random() < 0.8:
            masked_token = '<mask>'
        else:
            # 10%的时间：保持词不变
            if random.random() < 0.5:
                masked_token = tokens[mlm_pred_position]
            # 10%的时间：⽤随机词替换该词
            else:
                masked_token = random.choice(vocab.idx_to_token)
        mlm_input_tokens[mlm_pred_position] = masked_token
        pred_positions_and_labels.append(
            (mlm_pred_position, tokens[mlm_pred_position]))
    return mlm_input_tokens, pred_positions_and_labels

**通过调⽤前述的_replace_mlm_tokens函数，以下函数将BERT输⼊序列（tokens）作为输⼊，并返回输
⼊词元的索引、发⽣预测的词元索引以及这些预测的标签索引。**

In [20]:
def _get_mlm_data_from_tokens(tokens, vocab):
    candidate_pred_positions = []
    for i, token in enumerate(tokens):
        # 在遮蔽语言模型任务中不会预测特殊词元
        if token in ['<cls>', '<sep>']:
            continue
        candidate_pred_positions.append(i)
    # 遮蔽语言模型任务中预测15%的随机词元
    num_mlm_preds = max(1, round(len(tokens) * 0.15))
    mlm_input_tokens, pred_positions_and_labels = _replace_mlm_tokens(tokens, candidate_pred_positions, num_mlm_preds,
                                                                      vocab)
    pred_positions_and_labels = sorted(pred_positions_and_labels, key=lambda x: x[0])
    pred_positions = [v[0] for v in pred_positions_and_labels]
    mlm_pred_labels = [v[1] for v in pred_positions_and_labels]
    return vocab[mlm_input_tokens], pred_positions, vocab[mlm_pred_labels]

#### 将文本转换为预训练数据集现
**现在我们几乎准备好为BERT预训练定制一个Dataset类。在此之前， 我 们 仍 然 需 要 定 义 辅 助 函
数_pad_bert_inputs来将特殊的$'<mask>'$词元附加到输入。它的参数examples包含来自两个预训练
任务的辅助函数_get_nsp_data_from_paragraph和_get_mlm_data_from_tokens的输出。**

In [23]:
def _pad_bert_inputs(examples, max_len, vocab):
    max_num_mlm_preds = round(max_len * 0.15)
    all_token_ids, all_segments, valid_lens = [], [], []
    all_pred_positions, all_mlm_weights, all_mlm_labels = [], [], []
    nsp_labels = []
    for token_ids, pred_positions, mlm_pred_label_ids, segments, is_next in examples:
        all_token_ids.append(torch.tensor(token_ids + [vocab['<pad>']] * (max_len - len(token_ids)), dtype=torch.long))
        all_segments.append(torch.tensor(segments + [0] * (
                max_len - len(segments)), dtype=torch.long))
        # valid_lens不包括'<pad>'的计数
        valid_lens.append(torch.tensor(len(token_ids), dtype=torch.float32))
        all_pred_positions.append(
            torch.tensor(pred_positions + [0] * (max_num_mlm_preds - len(pred_positions)), dtype=torch.long))
        # 填充词元的预测将通过乘以0权重在损失中过滤掉
        all_mlm_weights.append(
            torch.tensor([1.0] * len(mlm_pred_label_ids) + [0.0] * (max_num_mlm_preds - len(pred_positions)),
                         dtype=torch.float32))
        all_mlm_labels.append(
            torch.tensor(mlm_pred_label_ids + [0] * (max_num_mlm_preds - len(mlm_pred_label_ids)), dtype=torch.long))
        nsp_labels.append(torch.tensor(is_next, dtype=torch.long))
    return all_token_ids, all_segments, valid_lens, all_pred_positions, all_mlm_weights, all_mlm_labels, nsp_labels

**将⽤于⽣成两个预训练任务的训练样本的辅助函数和⽤于填充输⼊的辅助函数放在⼀起，我们定义以
下_WikiTextDataset类为⽤于预训练BERT的WikiText-2数据集。通过实现__getitem__函数，我们可
以任意访问WikiText-2语料库的⼀对句⼦⽣成的预训练样本（遮蔽语⾔模型和下⼀句预测）样本。**

**最初的BERT模型使⽤词表⼤⼩为30000的WordPiece嵌⼊ [Wu et al., 2016]。WordPiece的词元化⽅法是对
14.6.2节中原有的字节对编码算法稍作修改。为简单起⻅，我们使⽤d2l.tokenize函数进⾏词元化。出
现次数少于5次的不频繁词元将被过滤掉。**

In [14]:
class _WikiTextDataset(torch.utils.data.Dataset):
    def __init__(self, paragraphs, max_len):
        # 输入paragraphs[i]是代表段落的句子字符串列表；
        # 而输出paragraphs[i]是代表段落的句子列表，其中每个句子都是词元列表
        paragraphs = [d2l.tokenize(
            paragraph, token='word') for paragraph in paragraphs]
        sentences = [sentence for paragraph in paragraphs
                     for sentence in paragraph]

        self.vocab = d2l.Vocab(sentences, min_freq=5, reserved_tokens=[
            '<pad>', '<mask>', '<cls>', '<sep>'])
        # 获取下一句子预测任务的数据
        examples = []
        for paragraph in paragraphs:
            examples.extend(_get_nsp_data_from_paragraph(paragraph, paragraphs, vocab=self.vocab, max_len=max_len))
        # 获取遮蔽语言模型任务的数据
        examples = [(_get_mlm_data_from_tokens(tokens, self.vocab) + (segments, is_next)) for tokens, segments, is_next
                    in examples]
        # 填充输入
        (self.all_token_ids, self.all_segments, self.valid_lens, self.all_pred_positions, self.all_mlm_weights,
         self.all_mlm_labels, self.nsp_labels) = _pad_bert_inputs(examples, max_len, self.vocab)

    def __getitem__(self, idx):
        return (self.all_token_ids[idx], self.all_segments[idx], self.valid_lens[idx], self.all_pred_positions[idx],
                self.all_mlm_weights[idx], self.all_mlm_labels[idx], self.nsp_labels[idx])

    def __len__(self):
        return len(self.all_token_ids)

**通过使用_read_wiki函数和_WikiTextDataset类，我们定义了下面的load_data_wiki来下载并生成WikiText-2数据集，并从中生成预训练样本。**

In [6]:
def load_data_wiki(batch_size, max_len):
    """加载WikiText-2数据集"""
    num_workers = d2l.get_dataloader_workers()
    data_dir = d2l.download_extract('wikitext-2', 'wikitext-2')
    paragraphs = _read_wiki(data_dir)
    train_set = _WikiTextDataset(paragraphs, max_len)
    train_iter = torch.utils.data.DataLoader(train_set, batch_size, shuffle=True, num_workers=num_workers)
    return train_iter, train_set.vocab

**将批量⼤⼩设置为512，将BERT输⼊序列的最⼤⻓度设置为64，我们打印出⼩批量的BERT预训练样本的形
状。注意，在每个BERT输⼊序列中，为遮蔽语⾔模型任务预测10（64 × 0.15）个位置。**

In [ ]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)
for (tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X, mlm_Y, nsp_y) in train_iter:
    print(tokens_X.shape, segments_X.shape, valid_lens_x.shape, pred_positions_X.shape, mlm_weights_X.shape,
          mlm_Y.shape, nsp_y.shape)
    break

## 预训练BERT
**利⽤实现的BERT模型和从WikiText-2数据集生成的预训练样本，我们将在本节中
在WikiText-2数据集上对BERT进行预训练。**

In [12]:
import torch
from torch import nn
from d2l import torch as d2l

In [24]:
batch_size, max_len = 512, 64
train_iter, vocab = load_data_wiki(batch_size, max_len)

#### 预训练BERT

**原始BERT [Devlin et al., 2018]有两个不同模型尺寸的版本。基本模型（BERTBASE）使用12层（Transformer编
码器块），768个隐藏单元（隐藏大小）和12个自注意头。大模型（BERTLARGE）使用24层，1024个隐藏单元
和16个自注意头。值得注意的是，前者有1.1亿个参数，后者有3.4亿个参数。为了便于演示，我们定义了一个
小的BERT，使用了2层、128个隐藏单元和2个自注意头。**

In [25]:
net = d2l.BERTModel(len(vocab), num_hiddens=128, norm_shape=[128], ffn_num_input=128, ffn_num_hiddens=256, num_heads=2,
                    num_layers=2, dropout=0.2, key_size=128, query_size=128, value_size=128, hid_in_features=128,
                    mlm_in_features=128, nsp_in_features=128)
device = d2l.try_all_gpus()
loss = nn.CrossEntropyLoss()

**在定义训练代码实现之前，我们定义了一个辅助函数_get_batch_loss_bert。给定训练样本，该函数计
算遮蔽语言模型和下⼀句⼦预测任务的损失。请注意，BERT预训练的最终损失是遮蔽语言模型损失和下一句预测损失的和。**

In [28]:
def _get_batch_loss_bert(net, loss, vocab_size, tokens_X, segments_X, valid_lens_x, pred_positions_X, mlm_weights_X,
                         mlm_Y, nsp_y):
    # 前向传播
    _, mlm_Y_hat, nsp_Y_hat = net(tokens_X, segments_X, valid_lens_x.reshape(-1), pred_positions_X)
    # 计算遮蔽语言模型损失
    mlm_l = loss(mlm_Y_hat.reshape(-1, vocab_size), mlm_Y.shape(-1)) * mlm_weights_X.reshape(-1, 1)
    # 计算下一句子预测任务的损失
    nsp_l = loss(nsp_Y_hat, nsp_y)
    l = mlm_l + nsp_l
    return mlm_l, nsp_l, l